In [130]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime
import ast

from sklearn.model_selection import train_test_split 

# BOG EXPERIMENT

In [131]:
clientes = pd.read_csv('BOG_2022-12-16_CLIENTES_proyecto_elasticidad.csv')
clientes

,grupo,lista
0,test,"[38383201, 47688647, 45173155, 76826945, 19394..."
1,control,"[79656013, 240120183, 199896419, 213072506, 13..."
2,all_users,[ 15956661 199619214 258626724 ... 239549106 ...


In [132]:
test = ast.literal_eval(clientes.lista[0])
control = ast.literal_eval(clientes.lista[1])
#all_users = ast.literal_eval(clientes.lista[2])

In [133]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BOG'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [134]:
query = """
SELECT DISTINCT
    s.site_identifier_value as region_code,
    EXTRACT(WEEK FROM ffg.close_date) AS week,
    bo.customer_id,
    bs.addl_product_id as padre_sku_id,
    bs2.name AS card_name,
    bcat2.name AS category,
    bcat.name AS subcat,
    baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} ) AS gmv_usd,
    SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    SUM ( boi.quantity* foi.step_unit ) AS cant,
    COUNT (DISTINCT bo.order_id) AS ordenes

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(ffg.close_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3,4,5,6,7,8
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd", "cant", "discount_applied"]] = data_ventas_aux[["gmv_usd", "cant", "discount_applied"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [135]:
data_ventas = data_ventas_aux[((data_ventas_aux.adjustment_reason == "SH_Proyecto_Elasticidad2022-12-16_BOG") | 
                 (data_ventas_aux.adjustment_reason.str.contains("Hook"))) & 
                 (data_ventas_aux.padre_sku_id.isin([204584,78522])) &
                 (data_ventas_aux.week == 51)].copy()
data_ventas

,region_code,week,customer_id,padre_sku_id,card_name,category,subcat,adjustment_reason,gmv_usd,discount_applied,cant,ordenes
714,BOG,51,70664948,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,86.087924,3.443517,2.0,1
794,BOG,51,45176094,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,43.260064,1.730403,1.0,1
970,BOG,51,139856589,78522,Harina de trigo El Lobo,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,664.973517,26.598941,15.0,1
1261,BOG,51,85281925,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,301.307733,12.052309,7.0,2
1451,BOG,51,69106205,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,129.780191,5.191207,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
390479,BOG,51,22008990,78522,Harina de trigo El Lobo,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,89.558792,3.582352,2.0,1
391562,BOG,51,186066820,78522,Harina de trigo El Lobo,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,134.338188,5.373527,3.0,1
392738,BOG,51,161906882,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,SH_Proyecto_Elasticidad2022-12-16_BOG,129.780191,5.191207,3.0,1
394991,BOG,51,17456239,204584,Harina de trigo indupan panificacion x 50 kg,Abarrotes,Harinas & Mezclas,Hook_BOG_Harina_de_trigo_indupan_panificacion_...,172.175847,3.443517,4.0,1


In [136]:
len(test)

58

In [137]:
print("ESTE ES EL GRUPO PILOTO (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL HOOK A LA MITAD)")
data_ventas[data_ventas.customer_id.isin(test)].groupby(['week']).sum().reset_index().iloc[[0],[0,3,4,5,6]]

ESTE ES EL GRUPO PILOTO (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL HOOK A LA MITAD)


,week,gmv_usd,discount_applied,cant,ordenes
0,51,5684.475367,118.376712,130.0,33


In [138]:
print("ESTE ES EL GRUPO CONTROL (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL MISMO HOOK)")
data_ventas[data_ventas.customer_id.isin(control)].groupby(['week']).sum().reset_index().iloc[[0],[0,3,4,5,6]]

ESTE ES EL GRUPO CONTROL (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL MISMO HOOK)


,week,gmv_usd,discount_applied,cant,ordenes
0,51,5061.320971,202.452837,117.0,35


# CMX EXPERIMENT

In [139]:
clientes = pd.read_csv('CMX_2022-12-16_CLIENTES_proyecto_elasticidad.csv')
clientes

,grupo,lista
0,test,"[78132730, 60537992, 226057944, 221400422, 111..."
1,control,"[76001797, 245100945, 214978459, 71822703, 124..."
2,all_users,[ 45907825 53810120 160128718 ... 202273383 ...


In [140]:
test_MX = ast.literal_eval(clientes.lista[0])
control_MX = ast.literal_eval(clientes.lista[1])
#all_users = ast.literal_eval(clientes.lista[2])

In [141]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [142]:
query = """
SELECT DISTINCT
    s.site_identifier_value as region_code,
    EXTRACT(WEEK FROM ffg.close_date) AS week,
    bo.customer_id,
    bs.addl_product_id as padre_sku_id,
    bs2.name AS card_name,
    bcat2.name AS category,
    bcat.name AS subcat,
    baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} ) AS gmv_usd,
    SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    SUM ( boi.quantity* foi.step_unit ) AS cant,
    COUNT (DISTINCT bo.order_id) AS ordenes

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(ffg.close_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3,4,5,6,7,8
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

data_ventas_aux_MX = read_connection_data_warehouse.runQuery(query)
data_ventas_aux_MX[["gmv_usd", "cant", "discount_applied"]] = data_ventas_aux_MX[["gmv_usd", "cant", "discount_applied"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [143]:
data_ventas_MX = data_ventas_aux_MX[((data_ventas_aux_MX.adjustment_reason == "SH_Proyecto_Elasticidad2022-12-16_CMX") | 
                 (data_ventas_aux_MX.adjustment_reason.str.contains("Hook"))) & 
                 (data_ventas_aux_MX.padre_sku_id.isin([55148])) &
                 (data_ventas_aux_MX.week == 51)].copy()
data_ventas_MX

,region_code,week,customer_id,padre_sku_id,card_name,category,subcat,adjustment_reason,gmv_usd,discount_applied,cant,ordenes
5,CMX,51,205726514,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,52.20610,2.08804,1.0,1
126,CMX,51,-265901,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,576.99745,25.71653,11.0,2
293,CMX,51,133228599,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,52.20610,2.08804,1.0,1
469,CMX,51,15060229,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,52.20610,2.08804,1.0,1
513,CMX,51,24188304,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,155.60050,7.25750,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
320684,CMX,51,148109049,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,52.75216,2.63765,1.0,1
320940,CMX,51,191652116,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,310.18320,15.50838,6.0,1
321028,CMX,51,-269422,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,105.50432,5.27530,2.0,1
321155,CMX,51,167782076,55148,Azúcar Morena Estándar de 50 Kg,Abarrotes,Azúcar & Endulzantes,SH_Proyecto_Elasticidad2022-12-16_CMX,51.69720,2.58473,1.0,1


In [144]:
len(test_MX)

93

In [145]:
print("ESTE ES EL GRUPO PILOTO (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL HOOK A LA MITAD)")
data_ventas_MX[data_ventas_MX.customer_id.isin(test_MX)].groupby(['week']).sum().reset_index().iloc[[0],[0,3,4,5,6]]

ESTE ES EL GRUPO PILOTO (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL HOOK A LA MITAD)


,week,gmv_usd,discount_applied,cant,ordenes
0,51,5008.18679,103.32535,96.02,39


In [146]:
print("ESTE ES EL GRUPO CONTROL (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL MISMO HOOK)")
data_ventas_MX[data_ventas_MX.customer_id.isin(control_MX)].groupby(['week']).sum().reset_index().iloc[[0],[0,3,4,5,6]]

ESTE ES EL GRUPO CONTROL (LOS QUE COMPRAN 6 VECES EN LAS ULTIMAS 10 SEMANAS, PERO CON EL MISMO HOOK)


,week,gmv_usd,discount_applied,cant,ordenes
0,51,6215.63932,277.89233,119.0,53
